In [1]:
import os
from functools import partial

from IPython.display import display, clear_output, SVG
import numpy as np
import pydot

import pydrake
from pydrake.all import (
    Adder, AddMultibodyPlantSceneGraph, Box, Cylinder, DiagramBuilder, 
    InverseKinematics, MeshcatVisualizerCpp, MeshcatVisualizerParams,
    Parser, FindResourceOrThrow, PassThrough,
    RigidTransform, RollPitchYaw, RotationMatrix, Simulator, Solve, Sphere,
    ConnectPlanarSceneGraphVisualizer, Demultiplexer, MultibodyPlant, PiecewiseQuaternionSlerp,
    InverseDynamicsController, StateInterpolatorWithDiscreteDerivative, TrajectorySource,
    LeafSystem, BasicVector, JacobianWrtVariable, RollPitchYaw, VectorLogSink, SpatialInertia, UnitInertia, Box, CoulombFriction,
    MathematicalProgram, Solve, eq, le, ge
)
from pydrake.examples.manipulation_station import ManipulationStation
from manipulation.meshcat_cpp_utils import (
    StartMeshcat, MeshcatPoseSliders, AddMeshcatTriad
)
from manipulation.scenarios import (
    AddIiwa, AddPlanarIiwa, AddTwoLinkIiwa, AddWsg, AddShape
)
from manipulation.utils import FindResource
from manipulation import running_as_notebook
from pydrake.trajectories import PiecewisePolynomial
from pydrake.multibody import inverse_kinematics
from pydrake.systems.controllers import PidController
from pydrake.systems.sensors import ImageToLcmImageArrayT
from pydrake.systems.sensors import PixelType
from pydrake.systems.lcm import LcmPublisherSystem
from pydrake.lcm import DrakeLcm
from pydrake.geometry import IllustrationProperties, SourceId, GeometryId, Rgba, PerceptionProperties
from pydrake.geometry.render import RenderLabel, RenderEngineVtkParams, DepthRenderCamera, RenderCameraCore, DepthRange
from pydrake.examples.mesh_painter import MakeRenderEngineVtkWithMaskImages, MeshPainterSystem
import time
import copy

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

In [3]:
# Task and Optimization Approach deffinitions and settings for Current Experiment

from enum import Enum

class Task(Enum):
    single_dirt_table = 1
    single_dirt_plate = 2
    multi_dirt_table = 3
    multi_dirt_plate = 4

class Approach(Enum):
    baseline = 0
    strict_constraint = 1
    threshold_constraint = 2
    min_distance_cost = 3

print([task for task in Task])
print([approach for approach in Approach])

CURRENT_TASK = Task.single_dirt_plate
CURRNET_APPROACH = Approach.strict_constraint

NUM_DIRT_PARTICLES = 5

if CURRENT_TASK == Task.single_dirt_table or CURRENT_TASK == Task.single_dirt_plate:
    NUM_DIRT_PATCHES = 1
else:
    # Set if multi task
    NUM_DIRT_PATCHES = 3

[<Task.single_dirt_table: 1>, <Task.single_dirt_plate: 2>, <Task.multi_dirt_table: 3>, <Task.multi_dirt_plate: 4>]
[<Approach.baseline: 0>, <Approach.strict_constraint: 1>, <Approach.threshold_constraint: 2>, <Approach.min_distance_cost: 3>]


In [4]:
# Function to setup and solve TrajOpt problem to generate wiping locations on table

def create_optimized_wipe_traj(dirt_locs, start_pos, end_pos, n):
    if CURRNET_APPROACH == Approach.baseline:
        traj = [start_pos] + [d for d in dirt_locs] + [end_pos]
        return np.array([xy[0] for xy in traj] + [xy[1] for xy in traj])

    prog = MathematicalProgram()

    # Create optimization variables for the x and y positions that we will command the 
    # arm to
    x_locs = prog.NewContinuousVariables(n, "x")
    y_locs = prog.NewContinuousVariables(n, "y")

    if CURRNET_APPROACH == Approach.strict_constraint:
        # Add a squared-distance cost to the points to incentivize the agent to not take a massively 
        # deviating trajectory
        omicron = 1
        threshold = 0.0001
        xy_vel = np.array([[x_locs[i+1] - x_locs[i], y_locs[i+1] - y_locs[i]] for i in range(n-1)])
        prog.AddCost(omicron*np.sum(np.square(xy_vel)))

        def vec_min(x):
            min_distance = x[0]
            for i in range(1, n):
                min_distance = pydrake.math.min(min_distance, x[i])

            return min_distance

        def min_distance_constraint(xy_locs, d_i):
            x_locs = xy_locs[:n]
            y_locs = xy_locs[n:]
            return vec_min([((dirt_locs[d_i,0]-x_locs[i])**2 + (dirt_locs[d_i,1]-y_locs[i])**2) for i in range(n)])


        for d_i in range(dirt_locs.shape[0]):
            def constraint(vars):
                min_distance = min_distance_constraint(vars, d_i)
                return np.array([min_distance])

            prog.AddConstraint(le(constraint(x_locs.tolist() + y_locs.tolist()), threshold))

        # Constrain the solution such that the first and last positions match start_pos and end_pos
        prog.AddConstraint(x_locs[0] == start_pos[0])
        prog.AddConstraint(y_locs[0] == start_pos[1])
        prog.AddConstraint(x_locs[-1] == end_pos[0])
        prog.AddConstraint(y_locs[-1] == end_pos[1])
        prog.AddConstraint(ge(x_locs, np.zeros(n)))
        prog.AddConstraint(le(x_locs, np.ones(n) * 2.0))
        prog.AddConstraint(ge(y_locs, np.ones(n) * -0.7))
        prog.AddConstraint(le(y_locs, np.ones(n) * 0.7))


    elif CURRNET_APPROACH == Approach.threshold_constraint:
        # Add a squared-distance cost to the points to incentivize the agent to not take a massively 
        # deviating trajectory
        omicron = 1
        threshold = 0.0005
        xy_vel = np.array([[x_locs[i+1] - x_locs[i], y_locs[i+1] - y_locs[i]] for i in range(n-1)])
        prog.AddCost(omicron*np.sum(np.square(xy_vel)))

        def vec_min(x):
            min_distance = x[0]
            for i in range(1, n):
                min_distance = pydrake.math.min(min_distance, x[i])

            return min_distance

        def min_distance_constraint(xy_locs, d_i):
            x_locs = xy_locs[:n]
            y_locs = xy_locs[n:]
            return vec_min([((dirt_locs[d_i,0]-x_locs[i])**2 + (dirt_locs[d_i,1]-y_locs[i])**2) for i in range(n)])


        for d_i in range(dirt_locs.shape[0]):
            def constraint(vars):
                min_distance = min_distance_constraint(vars, d_i)
                return np.array([min_distance])

            prog.AddConstraint(le(constraint(x_locs.tolist() + y_locs.tolist()), threshold))

        # Constrain the solution such that the first and last positions match start_pos and end_pos
        prog.AddConstraint(x_locs[0] == start_pos[0])
        prog.AddConstraint(y_locs[0] == start_pos[1])
        prog.AddConstraint(x_locs[-1] == end_pos[0])
        prog.AddConstraint(y_locs[-1] == end_pos[1])
        prog.AddConstraint(ge(x_locs, np.zeros(n)))
        prog.AddConstraint(le(x_locs, np.ones(n) * 2.0))
        prog.AddConstraint(ge(y_locs, np.ones(n) * -0.7))
        prog.AddConstraint(le(y_locs, np.ones(n) * 0.7))

    elif CURRNET_APPROACH == Approach.min_distance_cost:
        # Add a squared-distance cost to the points to incentivize the agent to not take a massively 
        # deviating trajectory
        omicron = 1
        xy_vel = np.array([[x_locs[i+1] - x_locs[i], y_locs[i+1] - y_locs[i]] for i in range(n-1)])
        prog.AddCost(omicron*np.sum(np.square(xy_vel)))

        def vec_min(x):
            min_distance = x[0]
            for i in range(1, n):
                min_distance = pydrake.math.min(min_distance, x[i])

            return min_distance

        def min_distance_cost(xy_locs):
            x_locs = xy_locs[:n]
            y_locs = xy_locs[n:]
            return sum([vec_min([((dirt_locs[j][0]-x_locs[i])**2 + (dirt_locs[j][1]-y_locs[i])**2) for i in range(n)]) for j in range(dirt_locs.shape[0])])


        prog.AddCost(min_distance_cost, vars=x_locs.tolist() + y_locs.tolist())

        # Constrain the solution such that the first and last positions match start_pos and end_pos
        prog.AddConstraint(x_locs[0] == start_pos[0])
        prog.AddConstraint(y_locs[0] == start_pos[1])
        prog.AddConstraint(x_locs[-1] == end_pos[0])
        prog.AddConstraint(y_locs[-1] == end_pos[1])
        prog.AddConstraint(ge(x_locs, np.zeros(n)))
        prog.AddConstraint(le(x_locs, np.ones(n) * 2.0))
        prog.AddConstraint(ge(y_locs, np.ones(n) * -0.7))
        prog.AddConstraint(le(y_locs, np.ones(n) * 0.7))


    prog.SetInitialGuess(x_locs, np.random.rand(n))
    prog.SetInitialGuess(y_locs, np.random.rand(n))
    
    print(prog)
    result = Solve(prog)

    # 6. Get the solution
    if (result.is_success()): 
        sol = result.GetSolution()
        x_sol = sol[0:n]
        y_sol = sol[n:]
        print(f"X Solution: {x_sol}")
        print(f"Y Solution: {y_sol}")
        return sol
    else:
        print("Failed to Solve :(")

In [5]:
# Randomly genrates a problem instance with random dirt locations on the table and generates a trajectory using prespecified optimization approahce

# Randomly generate dirt locations
max_x = 0.75
min_x = 0.35
max_y = 0.35
min_y = -0.35

print(NUM_DIRT_PATCHES)
#dirt_locs = np.array([[1, 0.1], [0.6, 0.1], [0.55, 0.0]])[:NUM_DIRT_PATCHES] #np.array([[0.8, 0.1], [0.65, 0.4], [0.55, 0.0]])
dirt_locs = np.stack([np.random.uniform(low=min_x, high=max_x, size=NUM_DIRT_PATCHES), np.random.uniform(low=min_y, high=max_y, size=NUM_DIRT_PATCHES)], axis=1)
dirt_locs[-1,0] = 0.7
dirt_locs[-1,1] = -0.3
#

start_pos = [0.6, 0.1]
end_pos = [0.55, 0.0]
n = 4
sol = None
while sol is None:
    sol = create_optimized_wipe_traj(dirt_locs=dirt_locs, start_pos=start_pos, end_pos=end_pos, n=n)
des_xypos_list = [np.array([sol[i], sol[int(len(sol)/2)+i]]) for i in range(int(len(sol)/2))]
print()
print(CURRNET_APPROACH)
print("Dirt Locations:", dirt_locs)
print("Trajectory:", des_xypos_list)

-inf <= x(2) <= 2
-inf <= x(3) <= 2
BoundingBoxConstraint
-0.7 <= y(0) <= inf
-0.7 <= y(1) <= inf
-0.7 <= y(2) <= inf
-0.7 <= y(3) <= inf
BoundingBoxConstraint
-inf <= y(0) <= 0.7
-inf <= y(1) <= 0.7
-inf <= y(2) <= 0.7
-inf <= y(3) <= 0.7

Failed to Solve :(
Decision variables:x(0) x(1) x(2) x(3) y(0) y(1) y(2) y(3)

QuadraticCost ((x(0) * (x(0) - x(1))) + (x(1) * ( - x(0) + 2 * x(1) - x(2))) + (x(2) * ( - x(1) + 2 * x(2) - x(3))) + (x(3) * ( - x(2) + x(3))) + (y(0) * (y(0) - y(1))) + (y(1) * ( - y(0) + 2 * y(1) - y(2))) + (y(2) * ( - y(1) + 2 * y(2) - y(3))) + (y(3) * ( - y(2) + y(3))))
ExpressionConstraint
-inf <= (-0.0001 + min(min(min((pow((-0.29999999999999999 - y(0)), 2) + pow((0.69999999999999996 - x(0)), 2)), (pow((-0.29999999999999999 - y(1)), 2) + pow((0.69999999999999996 - x(1)), 2))), (pow((-0.29999999999999999 - y(2)), 2) + pow((0.69999999999999996 - x(2)), 2))), (pow((-0.29999999999999999 - y(3)), 2) + pow((0.69999999999999996 - x(3)), 2)))) <= 0
LinearEqualityConstraint

In [6]:
# Converts from desired xy trajectory along the table to the bxyz trajectory we will feed into our control pipeline, 
# where b is a boolean that represents whether to use pure position control or hybrid-force postion control

print(dirt_locs)
print(des_xypos_list)

controller_trajectory = []
idx = 0
pos = des_xypos_list[idx]
clearance = 0.6
table_z = 0.24 #0.24499465
wipe_distance = 0.1
done = False
command = 0


for pos in des_xypos_list:
    # [0] GO TO DIRT LOCATION (Pos Control)
    controller_trajectory.append([1, pos[0], pos[1], clearance])

    # [1] DESCENT (Pos Control)
    if True in [(dirt_locs[d_i][0]-pos[0])**2 + (dirt_locs[d_i][1]-pos[1])**2 < 0.001 for d_i in range(dirt_locs.shape[0])]:
        controller_trajectory.append([1, pos[0], pos[1], table_z])

        # [2] WIPE (Force Control)
        # Forward
        controller_trajectory.append([0, pos[0] + wipe_distance, pos[1], table_z])
        # Back
        controller_trajectory.append([0, pos[0], pos[1], table_z])

        # [3] ASCENT(Pos Control)
        controller_trajectory.append([1, pos[0], pos[1], clearance])

print("Controller Trajectory")
print(controller_trajectory)

[[ 0.7 -0.3]]
[array([0.6, 0.1]), array([ 0.64814301, -0.09535763]), array([ 0.69628614, -0.29071521]), array([0.55, 0.  ])]
Controller Trajectory
[[1, 0.6, 0.09999999999999998, 0.6], [1, 0.64814301062726, -0.09535763115717223, 0.6], [1, 0.696286144976224, -0.2907152121811601, 0.6], [1, 0.696286144976224, -0.2907152121811601, 0.24], [0, 0.796286144976224, -0.2907152121811601, 0.24], [0, 0.696286144976224, -0.2907152121811601, 0.24], [1, 0.696286144976224, -0.2907152121811601, 0.6], [1, 0.55, 0.0, 0.6]]


In [7]:
# Verifies that trajectory reaches all generated dirt locations

def vec_min(x):
            min_distance = x[0]
            for i in range(1, n):
                min_distance = pydrake.math.min(min_distance, x[i])

            return min_distance

def min_distance_constraint(xy_locs, d_i):
    return vec_min([((dirt_locs[d_i][0]-xy_locs[i][0])**2 + (dirt_locs[d_i][1]-xy_locs[i][1])**2) for i in range(n)])

for d_i in range(dirt_locs.shape[0]):
    print(min_distance_constraint(des_xypos_list, d_i) < 0.001)

True


In [8]:
# IK solver definition for our controller to use

def create_q_knots(pose_lst, plant, angle_tol=0.0, xyz_tol=0.2):
  """Convert end-effector pose list to joint position list using series of
  InverseKinematics problems. q is 7-dimensional.
  @param: pose_lst (python list): post_lst[i] contains keyframe X_WG at index i.
  @return: q_knots (python_list): q_knots[i] contains IK solution that will give f(q_knots[i]) \approx pose_lst[i].
  """
  q_knots = []
  plant = plant
  world_frame = plant.world_frame()
  gripper_frame = plant.GetFrameByName("iiwa_link_7")
  q_nominal = np.array([0.08779705,  0.68502306,  0.00395995, -1.60894861, -0.00333842,  0.84763008, 0.09307069]) # nominal joint for joint-centering.

  def AddOrientationConstraint(ik, R_WG, bounds):
    """Add orientation constraint to the ik problem. Implements an inequality
    constraint where the axis-angle difference between f_R(q) and R_WG must be
    within bounds. Can be translated to:
    ik.prog().AddBoundingBoxConstraint(angle_diff(f_R(q), R_WG), -bounds, bounds)
    """
    ik.AddOrientationConstraint(
        frameAbar=world_frame, R_AbarA=R_WG,
        frameBbar=gripper_frame, R_BbarB=RotationMatrix(),
        theta_bound=bounds
    )

  def AddPositionConstraint(ik, p_WG_lower, p_WG_upper):
    """Add position constraint to the ik problem. Implements an inequality
    constraint where f_p(q) must lie between p_WG_lower and p_WG_upper. Can be
    translated to
    ik.prog().AddBoundingBoxConstraint(f_p(q), p_WG_lower, p_WG_upper)
    """
    ik.AddPositionConstraint(
        frameA=world_frame, frameB=gripper_frame, p_BQ=np.zeros(3),
        p_AQ_lower=p_WG_lower, p_AQ_upper=p_WG_upper)

  # print(f"pose list: {pose_lst}")
  for i in range(len(pose_lst)):
    ik = inverse_kinematics.InverseKinematics(plant)
    q_variables = ik.q() # Get variables for MathematicalProgram
    prog = ik.prog() # Get MathematicalProgram

    # Make sure gripper faces down
    AddOrientationConstraint(ik, RotationMatrix().MakeYRotation(-np.pi), angle_tol)

    # Make sure q's position is within some bound of pose for x and y axis
    AddPositionConstraint(ik,np.array([pose_lst[i].translation()[0]-xyz_tol, pose_lst[i].translation()[1]-xyz_tol, pose_lst[i].translation()[2]-xyz_tol]), np.array([pose_lst[i].translation()[0]+xyz_tol, pose_lst[i].translation()[1]+xyz_tol, pose_lst[i].translation()[2]+xyz_tol]))

    # print(f"Lower Bound: {np.array([pose_lst[i].translation()[0]-xyz_tol, pose_lst[i].translation()[1]-xyz_tol, pose_lst[i].translation()[2]-xyz_tol])}")
    # print(f"pose_list[{i}]: {pose_lst[i]}")

    prog.AddQuadraticErrorCost(np.identity(len(q_variables)), q_nominal, q_variables)

    if i==0:
      prog.SetInitialGuess(q_variables, q_nominal)
    else:
      prog.SetInitialGuess(q_variables, q_knots[-1][0:7])

    result = Solve(prog)

    if not result.is_success():
      raise RuntimeError
    else:
      q_solution = result.GetSolution(q_variables)
      if i==0:
        q_knots.append(np.concatenate([q_solution, np.zeros(7)], axis=0))
      else:
        q_knots.append(np.concatenate([q_solution, np.subtract(q_solution, q_knots[-1][0:7])], axis=0))

  # print(f"q_knots: {q_knots}")

  return q_knots

In [9]:
# Our controller definition

def compute_ctrl(p_xyz_now, v_xyz_now, rpy_now, px_des, py_des, fz_des, roll_des, pitch_des, yaw_des, pure_pos_control_mode):
  """Compute control action given current position and velocities, as well as 
  desired x-direction position p_des(t) / desired z-direction force f_des. 
  You may set theta_des yourself, though we recommend regulating it to zero. 
  Input:
    - p_xyz_now: np.array (dim 3), position of the EE. [px, py, pz] 
    - v_xyz_now: np.array (dim 6), velocity of the EE. [vroll, vpitch, vyaw, vx, vy, vz] 
    - rpy_now: np.array (dim 3), roll, pitch, yaw of the EE
    - px_des: float, desired x position
    - py_des: float, desired y position
    - fz_des: float, desired z force to exert
    - roll_des: float, desired roll in radians
    - pitch_des: float, desired pitch in radians
    - yaw_des: float, desired yaw in radians
    - pure_pos_control_mode: bool, true if we're in the pure position control setting of our manipulator (i.e, we're not in contact yet), 
      and false otherwise
  Output:
    - u: np.array (dim 6), spatial torques to send to the manipulator. [tau_x, tau_y, tau_z, fx, fy, fz]
    - reached_des: bool, True if the EE currently has reached (epsilon near) the desired x,y position. False otherwise 
  """
  u = np.zeros(6)
  reached_des = False
  epsilon = 0.06

  if not pure_pos_control_mode:
    # D roll
    kproll = 0.05
    kdroll = 2.0*np.sqrt(kproll)  # should really be kproll*I on inside
    # u[0] = kproll * (roll_des - rpy_now[0]) - kdroll*v_xyz_now[0]
    u[0] = -kdroll*v_xyz_now[0]

    # D pitch
    kppitch = 0.09
    kdpitch = 2.0*np.sqrt(kppitch)  # should really be kppitch*I on inside
    # u[1] = kppitch * (pitch_des - rpy_now[1]) - kdpitch*v_xyz_now[1]
    u[1] = - kdpitch*v_xyz_now[1]

    # D yaw
    kpyaw = 0.05
    kdyaw = 2.0*np.sqrt(kpyaw)  # should really be kpyaw*I on inside
    # u[2] = kpyaw * (yaw_des - rpy_now[2]) - kdyaw*v_xyz_now[2]
    u[2] = - kdyaw*v_xyz_now[2]

    # PD X
    kpx = 5.5
    kdx = 2.0*np.sqrt(kpx)  # should really be kpx*I on inside
    u[3] = kpx*(px_des - p_xyz_now[0]) - kdx*v_xyz_now[3]

    # PD Y
    kpy = 4.5
    kdy = 2.0*np.sqrt(kpy)  # should really be kpy*I on inside
    u[4] = kpy*(py_des - p_xyz_now[1]) - kdy*v_xyz_now[4]

    # PD Force Controller Z
    kpz = 1.2 #tune these
    kdz = 0.65 #tune these
    u[5] = -kpz * fz_des - kdz*v_xyz_now[5]

    if abs(px_des - p_xyz_now[0]) <= epsilon and abs(py_des - p_xyz_now[1]) <= epsilon:
      reached_des = True

  return u, reached_des

class TorqueController(LeafSystem):
  """Wrapper System for Commanding Pure Torques to planar iiwa.
    @param plant MultibodyPlant of the simulated plant.
    @param iiwa 
  """
  def __init__(self, plant, iiwa):
    LeafSystem.__init__(self)

    self._plant = plant 
    self._plant_context = plant.CreateDefaultContext() 
    self._iiwa = iiwa
    self._G = plant.GetFrameByName("iiwa_link_7", iiwa)
    self._W = plant.world_frame()
    self._ctrl_fun = compute_ctrl
    self._vx = -0.08125
    self._pure_pos_control_mode = True
    self._curr_trajectory_idx = 0
    self.controller_trajectory = controller_trajectory

    self.DeclareVectorInputPort("iiwa_position_measured", BasicVector(7))
    self.DeclareVectorInputPort("iiwa_velocity_measured", BasicVector(7))
    # self.DeclareVectorInputPort("iiwa_torque_external", BasicVector(7)) # This was added in to do force control on z

    self.DeclareVectorOutputPort("iiwa_position_command", BasicVector(14),
                                 self.CalcPositionOutput)
    self.DeclareVectorOutputPort("iiwa_torque_cmd", BasicVector(7),
                                 self.CalcTorqueOutput)
    self.DeclareVectorOutputPort("iiwa_torque_log", BasicVector(7),
                                 self.CalcTorqueOutputLog)

  def CalcPositionOutput(self, context, output):
    """Set q_d = q_now. This ensures the iiwa goes into pure torque mode in sim by setting the 
    position control torques in InverseDynamicsController to zero. 
    NOTE(terry-suh): Do not use this method on hardware or deploy this notebook on hardware. 
    We can only simulate pure torque control mode for iiwa on sim. 
    """
    # Get pure_pos_control_mode from index
    self._pure_pos_control_mode = self.controller_trajectory[self._curr_trajectory_idx][0]
    self._curr_desired_xyz_pos = self.controller_trajectory[self._curr_trajectory_idx][1:]

    # Below read from trajectory and increment next point
    # Position Control Trajectory to desired position self._curr_desired_xyz_pos #
    
    # Setup up constants
    speed = 2.5
    epsilon = 0.085

    # Read input ports
    q_now = self.get_input_port(0).Eval(context)
    v_now = self.get_input_port(1).Eval(context)
    self._plant.SetPositions(self._plant_context, self._iiwa, q_now)

    # 1. Retrieve the cartesian location of the gripper (i.e, the last iiwa link)
    context = self._plant_context
    gripper = self._plant.GetBodyByName("iiwa_link_7")
    current_pose = self._plant.EvalBodyPoseInWorld(context, gripper)
    rpy_now = RollPitchYaw(current_pose.rotation()).vector()
    p_xyz_now = current_pose.translation()

    if abs(self._curr_desired_xyz_pos[0] - p_xyz_now[0]) <= epsilon and abs(self._curr_desired_xyz_pos[1] - p_xyz_now[1]) and abs(self._curr_desired_xyz_pos[2] - p_xyz_now[2]) <= epsilon:
      reached_des = True
    else:
      reached_des = False

    if reached_des:
      if self._curr_trajectory_idx < len(self.controller_trajectory) - 1:
        self._curr_trajectory_idx += 1
      else:
        pass
      self._curr_desired_xyz_pos = self.controller_trajectory[self._curr_trajectory_idx][1:]
      self._pure_pos_control_mode = self.controller_trajectory[self._curr_trajectory_idx][0]
      print(p_xyz_now)
      print("(Position Controller) Moving on to next wiping position!")

    if self._pure_pos_control_mode:
      #clear_output(wait=True)
      print("[PC] Current_Position:", p_xyz_now)
      print(self._curr_desired_xyz_pos)
      print(self._curr_trajectory_idx,"/", len(self.controller_trajectory))
      # print("[PC] Current_RPY:", rpy_now)

      # Setup variables
      pose_lst = [current_pose]
      desired_pose = RigidTransform(current_pose.rotation(), self._curr_desired_xyz_pos)
      delta = np.subtract(desired_pose.translation(), current_pose.translation())

      # print("Using Position Control to move to desired wiping pose")
      if np.linalg.norm(delta) > 0.25: #1.0:
          scaled_delta = speed * delta #speed * delta / np.linalg.norm(delta)
      else:
          scaled_delta = delta
      
      next_pose = RigidTransform(current_pose.rotation(), np.add(pose_lst[-1].translation(), scaled_delta))
      
      pose_lst.append(next_pose)

      q_knots = np.array(create_q_knots(pose_lst, self._plant))
      q_d = q_knots[-1]

    else:
      # Set the joint poses to their current values, to simulate torque control
      q_d = np.concatenate([q_now, v_now/1.5]) #self.get_input_port(0).Eval(context)
    
    output.SetFromVector(q_d)

  def CalcTorqueOutput(self, context, output):
    # Get pure_pos_control_mode from index
    self._pure_pos_control_mode = self.controller_trajectory[self._curr_trajectory_idx][0]
    self._curr_desired_xyz_pos = self.controller_trajectory[self._curr_trajectory_idx][1:]

    if not self._pure_pos_control_mode:
      # Hard-coded position and force profiles. Can be connected from Trajectory class. 
      fz_des = 1.25
      # The desired roll, pitch and yaw are hardcoded to keep the gripper 'upright' w.r.t
      # the table so wiping can be accomplished correctly.
      roll_des = 3.13926408 
      pitch_des = 0.03596836
      yaw_des = -3.14142808

      # Read inputs 
      q_now = self.get_input_port(0).Eval(context)
      v_now = self.get_input_port(1).Eval(context)

      self._plant.SetPositions(self._plant_context, self._iiwa, q_now)

      # 1. Retrieve the cartesian location of the gripper (i.e, the last iiwa link)
      X_now = self._plant.CalcRelativeTransform(self._plant_context, self._W, self._G)
      rpy_now = RollPitchYaw(X_now.rotation()).vector()
      p_xyz_now = X_now.translation()

      print("[TC] Current_Position:", p_xyz_now)
      print(self._curr_desired_xyz_pos)
      print(self._curr_trajectory_idx,"/", len(self.controller_trajectory))
      
      # 2. Compute the Jacobian of the gripper frame w.r.t the robot arm base frame and use 
      # this to derive the cartesian velocity vector of the gripper frame
      J_G = self._plant.CalcJacobianSpatialVelocity(
          self._plant_context, JacobianWrtVariable.kQDot, 
          self._G, [0,0,0], self._W, self._W)
      v_xyz_now = J_G.dot(v_now)

      # 2. Apply ctrl_fun given the position and velocity of the robot to compute the 
      # EE-torque in Cartesian Space
      F_xyz, reached_des = self._ctrl_fun(p_xyz_now, v_xyz_now, rpy_now, self._curr_desired_xyz_pos[0], self._curr_desired_xyz_pos[1], fz_des, roll_des, pitch_des, yaw_des, self._pure_pos_control_mode)
      if reached_des:
        if self._curr_trajectory_idx < len(self.controller_trajectory) - 1:
          self._curr_trajectory_idx += 1
        else:
          pass
        self._curr_desired_xyz_pos = self.controller_trajectory[self._curr_trajectory_idx][1:]
        self._pure_pos_control_mode = self.controller_trajectory[self._curr_trajectory_idx][0]
        print(p_xyz_now)
        print(self._curr_desired_xyz_pos)
        print("(Torque Controller) Moving on to next wiping position!")

      # 3. Convert this torque back to joint space to apply a differential torque command!
      tau_cmd = J_G.T.dot(F_xyz)
    else:
      tau_cmd = np.zeros(7)
    output.SetFromVector(tau_cmd)
    return tau_cmd

  def CalcTorqueOutputLog(self, context, output):
    tau = self.CalcTorqueOutput(context, output)
    # print("tau:", tau)
    return tau

#####################################

In [ ]:
# Setup wiping bot simulation and run controller

def setup_wiping_bot(time_step=0.0002):
    builder = DiagramBuilder()

    # Setting up environment
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=time_step)
    parser = Parser(plant, scene_graph)
    iiwa = AddIiwa(plant)

    sponge_shape = Box(0.08, 0.065, 0.065)
    sponge_instance = AddShape(plant, sponge_shape, "sponge_link", mass=0.2, mu=0.1, color=[1.0, 1.0, 0.0, 1.0])
    
    X_7B = RigidTransform(RotationMatrix(), [0, 0, 0.068])
    plant.WeldFrames(plant.GetFrameByName("iiwa_link_7", iiwa), plant.GetFrameByName("sponge_link", sponge_instance), X_7B)


    # Task Dependent Objects #

    if CURRENT_TASK == Task.single_dirt_table:
        plates_to_wipe = None #parser.AddModelFromFile("plate.urdf", "plate")

        dirt_shape = pydrake.geometry.Box(0.005, 0.005, 0.005)

        number_dirt_patches = 1
        number_dirt_per_patch = NUM_DIRT_PARTICLES

        dirt = {}
        dirt["num_patches"] = number_dirt_patches
        dirt["num_per_patch"] = number_dirt_per_patch

        for i in range(number_dirt_patches):
            for j in range(number_dirt_per_patch):
                dirt[str(i) + "_" + str(j)] = AddShape(plant, dirt_shape, "dirt_" + str(i) + "_" + str(j), mass=0.001, mu=0.01, color=[0,0,0,1])

    elif CURRENT_TASK == Task.multi_dirt_table:
        plates_to_wipe = None #parser.AddModelFromFile("plate.urdf", "plate")

        dirt_shape = pydrake.geometry.Box(0.005, 0.005, 0.005)

        number_dirt_patches = NUM_DIRT_PATCHES
        number_dirt_per_patch = NUM_DIRT_PARTICLES

        dirt = {}
        dirt["num_patches"] = number_dirt_patches
        dirt["num_per_patch"] = number_dirt_per_patch

        for i in range(number_dirt_patches):
            for j in range(number_dirt_per_patch):
                dirt[str(i) + "_" + str(j)] = AddShape(plant, dirt_shape, "dirt_" + str(i) + "_" + str(j), mass=0.001, mu=0.1, color=[0,0,0,1])
    
    elif CURRENT_TASK == Task.single_dirt_plate:
        plates_to_wipe = {}

        dirt_shape = pydrake.geometry.Box(0.005, 0.005, 0.005)

        number_dirt_patches = 1
        number_dirt_per_patch = NUM_DIRT_PARTICLES

        dirt = {}
        dirt["num_patches"] = number_dirt_patches
        dirt["num_per_patch"] = number_dirt_per_patch

        for i in range(number_dirt_patches):
            plates_to_wipe[str(i)] = AddShape(plant, pydrake.geometry.Cylinder(0.1, 0.01), "plate_" + str(i), mass=0.5, mu=0.8, color=[1,1,1,1])
            for j in range(number_dirt_per_patch):
                dirt[str(i) + "_" + str(j)] = AddShape(plant, dirt_shape, "dirt_" + str(i) + "_" + str(j), mass=0.01, mu=0.1, color=[0,0,0,1])

    elif CURRENT_TASK == Task.multi_dirt_plate:
        plates_to_wipe = {}

        dirt_shape = pydrake.geometry.Box(0.005, 0.005, 0.005)

        number_dirt_patches = NUM_DIRT_PATCHES
        number_dirt_per_patch = NUM_DIRT_PARTICLES

        dirt = {}
        dirt["num_patches"] = number_dirt_patches
        dirt["num_per_patch"] = number_dirt_per_patch

        for i in range(number_dirt_patches):
            plates_to_wipe[str(i)] = AddShape(plant, pydrake.geometry.Cylinder(0.1, 0.01), "plate_" + str(i), mass=0.5, mu=0.8, color=[1,1,1,1])
            for j in range(number_dirt_per_patch):
                dirt[str(i) + "_" + str(j)] = AddShape(plant, dirt_shape, "dirt_" + str(i) + "_" + str(j), mass=0.01, mu=0.1, color=[0,0,0,1])

    ###
    table_base = AddShape(plant, pydrake.geometry.Box(0.80, 0.80, 0.15), "table_to_wipe_base", mass=15, mu=0.1, color=[0.59,0.29,0.0,1.0])
    X_0B = RigidTransform(RotationMatrix(), [0.65, 0, 0.070])
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("table_to_wipe_base", table_base), X_0B)
    plant.Finalize()

    # NOTE: Code taken from notebook setup by Prof. Russ Tedrake 
    # (https://deepnote.com/project/Manipulation-Station-p5S24ob_QPipeARm6RkGqQ/%2Fmanipulation_station.ipynb/#00003-c02ff458-1ca6-4059-9c48-a96ad5962c94)
    #########################################################################################################################
    num_iiwa_positions = plant.num_positions(iiwa)
    # I need a PassThrough system so that I can export the input port.
    iiwa_position = builder.AddSystem(PassThrough(num_iiwa_positions))
    # builder.ExportInput(iiwa_position.get_input_port(), "iiwa_position")
    builder.ExportOutput(iiwa_position.get_output_port(), "iiwa_position_command")

    # Export the iiwa "state" outputs.
    demux = builder.AddSystem(Demultiplexer(
        2 * num_iiwa_positions, num_iiwa_positions))
    builder.Connect(plant.get_state_output_port(iiwa), demux.get_input_port())
    builder.ExportOutput(demux.get_output_port(0), "iiwa_position_measured")
    builder.ExportOutput(demux.get_output_port(1), "iiwa_velocity_estimated")
    builder.ExportOutput(plant.get_state_output_port(iiwa), "iiwa_state_estimated")

    # Make the plant for the iiwa controller to use.
    controller_plant = MultibodyPlant(time_step=time_step)
    controller_iiwa = AddIiwa(controller_plant)
    controller_plant.Finalize()

    # Add the iiwa controller
    iiwa_controller = builder.AddSystem(
        InverseDynamicsController(
            controller_plant,
            kp=[100]*num_iiwa_positions,
            ki=[1]*num_iiwa_positions,
            kd=[20]*num_iiwa_positions,
            has_reference_acceleration=False))
    iiwa_controller.set_name("iiwa_controller")
    builder.Connect(
        plant.get_state_output_port(iiwa), iiwa_controller.get_input_port_estimated_state())

    # Add in the feed-forward torque
    adder = builder.AddSystem(Adder(2, num_iiwa_positions))
    builder.Connect(iiwa_controller.get_output_port_control(),
                    adder.get_input_port(0))
    # Use a PassThrough to make the port optional (it will provide zero values if not connected).
    torque_passthrough = builder.AddSystem(PassThrough([0]*num_iiwa_positions))
    builder.Connect(torque_passthrough.get_output_port(),
                    adder.get_input_port(1))
    # builder.ExportInput(torque_passthrough.get_input_port(), 
    #                     "iiwa_feedforward_torque")
    builder.Connect(adder.get_output_port(),
                    plant.get_actuation_input_port(iiwa))

    # Export "cheat" ports.
    builder.ExportOutput(scene_graph.get_query_output_port(), "geometry_query")
    builder.ExportOutput(plant.get_contact_results_output_port(), 
                         "contact_results")
    builder.ExportOutput(plant.get_state_output_port(), 
                         "plant_continuous_state")

    #########################################################################################################################

    # Add the TorqueController for the iiwa
    logger = builder.AddSystem(VectorLogSink(7))
    hfp_controller = builder.AddSystem(TorqueController(controller_plant, controller_iiwa))
    hfp_controller.set_name("hfp_controller")
    # builder.Connect(hfp_controller.get_output_port(0),
    #                 iiwa_position.get_input_port()) # This is the iiwa position input port
    builder.Connect(hfp_controller.get_output_port(0),
                    iiwa_controller.get_input_port_desired_state()) # This is the iiwa position input port
    builder.Connect(hfp_controller.get_output_port(1),
                    torque_passthrough.get_input_port()) # This is the iiwa feedforward torque port
    builder.Connect(hfp_controller.get_output_port(2),
                    logger.get_input_port(0))
    
    builder.Connect(demux.get_output_port(0), # This is the iiwa position measured port
                    hfp_controller.get_input_port(0))
    builder.Connect(demux.get_output_port(1), # This is the iiwa velocity estimated port
                    hfp_controller.get_input_port(1))

    visualizer = MeshcatVisualizerCpp.AddToBuilder(
        builder, 
        scene_graph, 
        meshcat,
        MeshcatVisualizerParams(delete_prefix_initialization_event=False))


    diagram = builder.Build()

    return diagram, plant, plates_to_wipe, dirt

    
wiping_bot_diagram, plant, plates_to_wipe, dirt = setup_wiping_bot()
context = wiping_bot_diagram.CreateDefaultContext()
wiping_bot_diagram.Publish(context)
simulator = Simulator(wiping_bot_diagram)

# Task Dependent Object Position Settings #

if CURRENT_TASK == Task.single_dirt_plate or CURRENT_TASK == Task.multi_dirt_plate:
    for i in range(dirt["num_patches"]):
        x_center = dirt_locs[i][0]
        y_center = dirt_locs[i][1]
        plant.SetPositions(plant.GetMyContextFromRoot(simulator.get_mutable_context()), 
            plates_to_wipe[str(i)],
            np.array([0.707, 0, 0, 0.707, x_center, y_center, 0.17]))

for i in range(dirt["num_patches"]):
    x_center = dirt_locs[i][0]
    y_center = dirt_locs[i][1]
    for j in range(dirt["num_per_patch"]):
        xy_pos = np.array([x_center, y_center]) + np.random.rand(2)*0.01 - 0.005
        if CURRENT_TASK == Task.single_dirt_table or CURRENT_TASK == Task.multi_dirt_table:
            dirt_location = np.array([0.707, 0, 0, 0.707, xy_pos[0], xy_pos[1], 0.16])
        else:
            dirt_location = np.array([0.707, 0, 0, 0.707, xy_pos[0], xy_pos[1], 0.25])
        plant.SetPositions(plant.GetMyContextFromRoot(simulator.get_mutable_context()), 
                                dirt[str(i) + "_" + str(j)],
                                dirt_location)

###
print(dirt_locs)
print(controller_trajectory)

simulator.set_target_realtime_rate(0)
simulator.AdvanceTo(100000.0)

[PC] Current_Position: [0.54379114 0.04432003 0.42083952]
[0.64814301062726, -0.09535763115717223, 0.6]
1 / 8
[PC] Current_Position: [0.54379114 0.04432003 0.42083952]
[0.64814301062726, -0.09535763115717223, 0.6]
1 / 8
[PC] Current_Position: [0.54379114 0.04432003 0.42083952]
[0.64814301062726, -0.09535763115717223, 0.6]
1 / 8
[PC] Current_Position: [0.54379132 0.04431988 0.42083979]
[0.64814301062726, -0.09535763115717223, 0.6]
1 / 8
[PC] Current_Position: [0.54379132 0.04431988 0.42083979]
[0.64814301062726, -0.09535763115717223, 0.6]
1 / 8
[PC] Current_Position: [0.54379132 0.04431988 0.42083979]
[0.64814301062726, -0.09535763115717223, 0.6]
1 / 8
[PC] Current_Position: [0.54379132 0.04431988 0.42083979]
[0.64814301062726, -0.09535763115717223, 0.6]
1 / 8
[PC] Current_Position: [0.54379149 0.04431974 0.42084004]
[0.64814301062726, -0.09535763115717223, 0.6]
1 / 8
[PC] Current_Position: [0.54379149 0.04431974 0.42084004]
[0.64814301062726, -0.09535763115717223, 0.6]
1 / 8
[PC] Curre

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e1f7a4de-0698-4427-bcc0-40d4d181e518' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>